# Monte Carlo
Begin: 2024-03-16

- 大多数的机器学习方法中使用的是参数的 **点估计 (point estimators)**，i.e.，$\arg \max_{\theta} f_\theta(X)$
- 在估计分布的参数时：极大似然估计(MLE)和最大后验估计(MAP)
- 在无法求得 $\arg \max_\theta f_\theta(X)$ 的解析解的情况下，可以采用一些迭代方法，如EM算法

这些方法都是点估计，因为它们只是给我们一个“最佳”的单一的 $\theta$。

## Monte Carlo Integration

### 1. 数学期望计算
(1). 要计算随机变量的某个函数的期望 $\mathbb E[f(x)]$，则需要计算以下积分：
    $$
        \mathbb E [f(x)] = \int f(x) \, p(x) \  \mathrm{d} x, \ x \in \mathbb R^n, \ f: \mathbb R^n \rightarrow \mathbb R^m
    $$
   $p(x)$ 是 $x$ 的目标分布，可以使用数值积分有效计算。
<br>

(2). 另一种方法是抽取多个随机样本，$x_n \sim p(x)$，然后计算：
    $$
        \mathbb E [f(x)]  \approx \frac{1}{N_S} \sum_{i=1}^{N_S} f(x_i)
    $$
   这称为**蒙特卡罗积分**。可以证明精度在原则上与 $x$ 维度无关，仅取决于样本数量 $N_S$。
<br>
### 2. 积分计算
对于一个一般的积分问题：$\int_x h(x) \ \mathrm{d} x$，可以找一个与积分问题的定义域空间一样的比较简单的分布 $X \sim P(X)$，并且满足 $\int_x p(x) \ \mathrm{d}x = 1$。采样 $n$ 个样本点 $\{x_1, x_2, \dots, x_n\}$，那么可以计算：
$$
    \int_x h(x) \ \mathrm{d}x = \int_x \frac{h(x)}{p(x)} \cdot p(x) \ \mathrm{d}x \approx \frac{1}{n} \sum_{i=1}^n \frac{h(x_i)}{p(x_i)}
$$
任何一个求积分的问题可以等价成求期望的问题。

在许多机器学习问题中，我们实际上对后验分布更加感兴趣：
$$
    p(\theta | \text{Data}) \propto p(\text{Data} | \theta) \, p(\theta)
$$
当我们需要得到这个后验分布的特征，如均值：$\mathbb E_{p(\theta | X)} [\theta]$，它的积分形式为：
$$
    \int_\theta \theta \, p(\theta | X) \ \mathrm{d} x
$$
但是当这个后验分布太过复杂时，我们无法计算积分，因此可以使用采样方法来 **近似** 这一均值：$\theta_i \sim\limits^{\text{iid}} p(\theta | X)$
$$
    \mathbb E_{p(\theta | X)} [\theta] \approx \frac{1}{n} \sum_{i=1}^n \theta_i
$$


### Example 1
**Q:** 求积分 $\int_0^1 e^{-x^2 / 2} \ \mathrm{d}x$。

**A:** 可以令 $p(x) = 1, \ x \sim  U(0, 1)$，在(0, 1)上进行均匀采样得到 $\{x_1, x_2, \dots, x_n\}$，然后计算：
$$
    \frac{1}{n} \sum_{i=1}^n \frac{h(x_i)}{p(x_i)} = \frac{1}{n} \sum_{i=1}^n h(x_i) = \frac{1}{n} \sum_{i=1}^n e^{-x_i^2 / 2}
$$

关于均匀分布采样的实现，可以使用 `np.random.random`。也可以从原理上实现：**线性同余**


In [1]:
import numpy as np
import matplotlib.pyplot as plt


def generate_random(size=None, a=9, c=3, m=1024, seed=0):
    rst = []
    v = seed
    for _ in range(0, size):
        v = (a * v + c) % m
        rst.append(v / m)
    return np.asarray(rst)


# 定义计算式
def func(x):
    return np.mean(np.exp(-0.5 * (x ** 2)))

In [2]:
print(func(np.random.random(10)))
print(func(np.random.random(100)))
print(func(np.random.random(10000)))
print(func(np.random.random(1000000)))

0.8136962966706813
0.8282664545917736
0.8560033934887978
0.8555805417838129


### Example 2

**Q:** 求积分 $\int_{- \infty}^{\infty} x \frac{1}{\sqrt{2\pi}} \exp{(\frac{-x^2}{2})} \ \mathrm{d}x$。

**A:** 可以令 $p(x) = \frac{1}{\sqrt{2\pi}} \exp{(\frac{-x^2}{2})}$，即 $p(x)$ 刚好是一个标准的正态分布，那么可以对 $p(x)$ 采样 $n$ 个样本 $\{x_1, x_2, \dots, x_n\}$，就可以计算其积分的近似值：
$$
    \frac{1}{n} \sum_{i=1}^n \frac{h(x_i)}{p(x_i)} = \frac{1}{n} \sum_{i=1}^n x_i
$$
标准的正态分布可以使用 `np.random.randn`，从原理上实现正态分布采样：**Box Muller**。